# Simulation of poromechanical processes in a fractured porous medium
This noteboook aims to give an overview of the problem of poromechanical deformation of
a fractured porous media, and how this can be simulated in 2d. 

Please note that we assume the reader has already gone through the notebook on
mixed-dimensional flow problems.


## Governing equations
PorePy simulates mechanical deformation of a porous media, denoted $\Omega_M$ as a coupled system with 
conservation of momentuum and mass. Using the displacement $u$ and the fluid pressure $p$
as primary variables, the conservation equations are 
$$\nabla \cdot \sigma = \nabla\cdot (\mathbb{C} \epsilon - I\alpha p_m) = f_s $$
for momentum and 
$$\frac{\partial \rho\phi}{\partial t} - \nabla\cdot (\rho K \nabla p_m) = f_f$$
for mass.

Here, $\sigma$ is the stress, $\mathbf{C}$ is the stiffness matrix, $\epsilon$ the
symmetrized displacement gradient, $I$ an identity tensor and $f_s$ body forces.
Moreover, $\rho$ represents fluid density, $\phi$ porosity, $K$ permeability,  
$f_f$ fluid sources and sinks, and the subscript $m$ on the pressure marks that this is the pressure in the matrix.
In addition come initial and boundary conditions; we will for simplicity disregard these.

Next, we introduce a fracture $\Omega_F$ and let $\Gamma$ represent the interface between $\Omega_M$ and $\Omega_F$.
The model for fluid flow in the fracture is the same as in the notebook for mixed
dimensional flow, and we will not review it here.
For the mechanical behavior, we introduce the contact stress $T$ as

$$T = n\cdot(\mathbb{C}\epsilon - I p_m) - p_f I \cdot n $$

Here, $n$ is a vector normal to the fracture.
The model for the deformation of the fracture consists of three components. 
First, balance of forces across the fractures, which reads

$$T^+ + T^- = 0$$

Where superscripts $+$ and $-$ denote the two sides of the fracture.
Second, in the normal direction of the fracture, there are two possible configurations:
Either, the fracture is open (the walls are not in contact) and the contact force is zero,
or the walls are in contact and the force is potentially non-zero.
Mathematically this can be expressed as

$$ [u]_n\geq 0 \qquad [u]_n T_n = 0 \qquad T_n \leq 0 $$

where subscript $n$ represents the normal component of a vector and $[u]$ is the displacement
jump across the fracture.

Third, motion in the tangential direction of the fracture is restricted by a Coulomb-type friction law:
There will be no motion as long as the tangential forces are insufficient to overcome the
frictional resistance, but if this is overcome, the displacement jump in the tangential
direction will change (note: we do not resolve the actual movement of the walls, but 
only search for the next static condition). This is expressed in terms of the following
equations:

$$

\begin{align}
    ||T_\tau||&\leq - F T_n \\
    ||T_\tau||&< - F T_n \Rightarrow [u]_\tau =0\\
    ||T_\tau||&= - F T_n \Rightarrow \exists \,\zeta \in \mathbb{R^+}: [u]_\tau = \zeta T_\tau.
\end{align}

$$

where $F$ represents the coefficient of friction and subscript $\tau$ denotes the
tangential component of a vector.

We see that this is a problem of considerabe difficulty: Not only is the problem non-linear,
which equations we have to solve actually depends on the solution. This is an example of
a class of problems known as variational inequalities. Fortunately, PorePy has implemented
methods to deal with these problems, and it is to implementational aspects we now turn.


## Implementation
To construct a PorePy simulation model for poromechanical fracture deformation, we proceed
in much the same way as we did for the flow problem, except that the template model
that we will expand on is `pp.poromechanics.Poromechanics`.

The first step is to define a geometry class that sets a fracture network. For this, we
simply copy the relevant function from the flow notebook.

In [1]:
# First the usual imports
import porepy as pp
import numpy as np

# We will use the same mesh as in the flow problem
class GeometryWithFractures(pp.ModelGeometry):
    # We need to make a class which inherits from the class pp.ModelGeometry (if you don't
    # know what inheritance means, you just have to accept this for now, but do google
    # inheritance in object oriented programing at some point).

    def set_fracture_network(self):
        # The class pp.ModelGeometry has several useful methods that we will leave
        # unaltered, but we do need to change the method which specifies the geometry.
        #
        # The code below is copied from above, except from a crucial modification in the
        # last line.
        #
        # PorePy has a built-in system to change units (for instance, measure
        # distance in milimeters rather than meters) and make sure this is done
        # consistently throughout the equations in a model. This typically requires
        # some boilerplate, like the one we use below. Strictly speaking, it is not
        # needed here, since our domain is of unit size, but it is good pracice to get
        # used to this anyhow.
        length_scaling = 1 / self.units.m

        # Define the domain. The easiest option here is to set a box-shaped domain,
        # specified by its min and max coordinates.
        domain = pp.Domain({'xmin': 0,
                            'xmax': 1 * length_scaling,
                            'ymin': 0,
                            'ymax': 1 * length_scaling
                            })

        # A line fracture is defined by its endpoints, specified in the form of a numpy list.append
        # The first row defines the x-coordinates, the second the y-coordinates, so this fracture
        # runs from (0.3, 0.4) to (0.7, 0.6).
        frac_1 = pp.LineFracture(np.array([[0.3, 0.7], [0.4, 0.6]]) * length_scaling)

        # We can define a second fracture. This will cross the first one in (0.5, 0.5)
        frac_2 = pp.LineFracture(np.array([[0.4, 0.6], [0.3, 0.7]]) * length_scaling,)

        # Add a third fracture which extends outside the domain
        frac_3 = pp.LineFracture(np.array([[0.8, 1.2], [0.8, 1.2]]) * length_scaling,)

        # Collect all fractures in a list
        fracture_set = [frac_1, frac_2, frac_3]

        # Define the fracture network, and assign it to the attribute
        # self.fracture_network
        self.fracture_network = pp.FractureNetwork2d(fracture_set, domain=domain)

    def mesh_arguments(self) -> dict:
        # We also need to set the mesh size arguments. These are also scaled with length
        length_scaling = 1 / self.units.m
        return {'mesh_size_min': 0.03 * length_scaling,
                'mesh_size_bound': 0.2 * length_scaling,
                'mesh_size_frac': 0.1 * length_scaling
                }

The default settings poromechanics model are much the same as for the flow problem in that
they do not give very interesting results. Neveretheless, we define a model class and
solve a problem, just so that we can make a contrast later on.

In [ ]:
class BoringPoroMechanics(GeometryWithFractures, pp.poromechanics.Poromechanics):
    pass

# Create a model object
boring_model = BoringPoroMechanics()
# Run the model
pp.run_time_dependent_model(boring_model, params={})
# Plot the pressure solution
pp.plot_grid(boring_model.mdg, 'pressure')

# We can also plot the displacement solution, but for that we need to work a little bit:
# The displacement is a vector
pp.plot_grid(boring_model.mdg, vector_value='u')

You may get a highly oscillating pressure field, but note the range of the color bars: In practice, these are all zero.

Next, let us change the boundary conditions for the mechanics. The template sets zero Dirichlet conditions (e.g., no displacement) on all sides, 
but we will change this to specifying forces on the right and top boundary.

In [ ]:
class ModifiedBoundaryCondition(pp.poromechanics.BoundaryConditionsPoromechanics):

    def bc_type_mechanics(self, sd: pp.Grid) -> pp.BoundaryConditionVectorial:
        # Copied from pp.momentum_balance.BoundaryConditionsMomentumBalance

        # Get hold of the domain boundary
        domain_boundary = self.domain_boundary_sides(sd)

        # Set Dirichlet boundary conditions on the left and bottom faces. The other
        # faces are Neumann by default.
        left_faces = domain_boundary.west
        bottom_faces = domain_boundary.south
        boundary_faces = np.hstack((left_faces, bottom_faces))
        bc = pp.BoundaryConditionVectorial(sd, boundary_faces, "dir")

        # Default internal BC is Neumann. We change to Dirichlet for the contact
        # problem. I.e., the mortar variable represents the displacement on the fracture
        # faces.
        # DO NOT CHANGE THIS! It will break the contact mechanics formulation.
        bc.internal_to_dirichlet(sd)
        return bc

    def bc_values_mechanics(self, subdomains: list[pp.Grid]) -> pp.ad.DenseArray:
        # This method should only be called by the top-dimensional domain, as this is
        # the only one that has such boundary conditions.
        assert len(subdomains) == 1
        sd = subdomains[0]

        # We plan to set forces on the right and top faces; on the other faces, the
        # boundary condition is zero.
        domain_boundary = self.domain_boundary_sides(sd)
        right_faces = domain_boundary.east
        top_faces = domain_boundary.north

        # Vector to store the boundary condition values. Note the dimension: For now
        # we represent it as a 2 x num_faces array, but it will be converted to a
        # 1d array of length 2 * num_faces before being returned.
        values = np.zeros((sd.dim, sd.num_faces))

        # The boundary condition is a vector, so we need to specify the values for
        # each component. Since we are setting Neuamn boundary conditions, we need to
        # set values that are scaled with the area of the face (think of this as setting
        # a total force on the face, rather than a force per unit area).
        # The sign convention is also important: A force is positive if it is directed
        # in the positive direction of the coordinate system. Here we want compression
        # on the right and top faces, so we set negative values.
        values[0, right_faces] = -sd.face_areas[right_faces]
        values[1, top_faces] = -sd.face_areas[top_faces]

        # Convert to a 1d array. The "F" argument means that the array is flattened
        # column-wise, rather than row-wise. In this way, the returned values first has
        # all components for the first face, then all components for the second face,
        # and so on.
        values = values.ravel("F")

        return pp.wrap_as_ad_array(values, "bc_vals_mechanics")
    

# Define a new class which also uses the modified boundary condition
class ModifiedPoromechanics(GeometryWithFractures,
                             ModifiedBoundaryCondition,
                               pp.poromechanics.Poromechanics):
    pass

# Create a model object
modified_model = ModifiedPoromechanics()
# Run the model
pp.run_time_dependent_model(modified_model, params={})
# Plot the pressure solution
pp.plot_grid(modified_model.mdg, 'pressure')

    